# Private Housing Resale Price Prediction
This file is dedicated to predicting private housing resale prices.

## Data Collection
- The following codes below in this section is dedicated to reading and combining csv files into 1 dataframe.
- We also import all necessary modules in this section as well.

In [10]:
# Necessary Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import datetime
import glob

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import *

In [11]:
# Read csv files for private housing data
privateHouseFile = "../dataset/sale-prediction/private-housing/*.csv"
all_private_sales = glob.glob(privateHouseFile)

listOfPrivate = []
for filename in all_private_sales:
    df = pd.read_csv(filename, skiprows=1)
    listOfPrivate.append(df)

privateDf = pd.concat(listOfPrivate, axis=0)
privateDf.reset_index(drop=True, inplace=True)
privateDf.head(500)

,S/N,Project Name,Street Name,Type,Postal District,Market Segment,Tenure,Type of Sale,No. of Units,Price ($),Nett Price ($),Area (Sqm),Type of Area,Floor Level,Unit Price ($psm),Date of Sale,Area (Sqft),Unit Price ($psf)
0,1,ECHELON,ALEXANDRA VIEW,Condominium,03,RCR,99 yrs lease commencing from 2012,Resale,1.0,2000000.0,-,93.0,Strata,41 to 45,21505.0,May-2021,NaN,NaN
1,2,CARIBBEAN AT KEPPEL BAY,KEPPEL BAY DRIVE,Condominium,04,RCR,99 yrs lease commencing from 1999,Resale,1.0,2300000.0,-,141.0,Strata,01 to 05,16312.0,May-2021,NaN,NaN
2,3,LUMIERE,MISTRI ROAD,Apartment,02,CCR,99 yrs lease commencing from 2006,Resale,1.0,1251300.0,-,60.0,Strata,36 to 40,20855.0,May-2021,NaN,NaN
3,4,ASCENTIA SKY,ALEXANDRA VIEW,Condominium,03,RCR,99 yrs lease commencing from 2008,Resale,1.0,2530000.0,-,169.0,Strata,16 to 20,14970.0,May-2021,NaN,NaN
4,5,HUNDRED TREES,WEST COAST DRIVE,Condominium,05,OCR,956 yrs lease commencing from 1928,Resale,1.0,1150000.0,-,73.0,Strata,01 to 05,15753.0,May-2021,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496,DOVER PARKVIEW,DOVER RISE,Condominium,05,RCR,99 yrs lease commencing from 1993,Resale,1.0,1050000.0,-,87.0,Strata,11 to 15,12069.0,Mar-2021,NaN,NaN
496,497,ONE SHENTON,SHENTON WAY,Apartment,01,CCR,99 yrs lease commencing from 2005,Resale,1.0,1390000.0,-,77.0,Strata,11 to 15,18052.0,Mar-2021,NaN,NaN
497,498,ALEX RESIDENCES,ALEXANDRA VIEW,Apartment,03,RCR,99 yrs lease commencing from 2013,Resale,1.0,1270000.0,-,61.0,Strata,16 to 20,20820.0,Mar-2021,NaN,NaN
498,499,MARINA ONE RESIDENCES,MARINA WAY,Apartment,01,CCR,99 yrs lease commencing from 2011,Resale,1.0,2471812.0,-,96.0,Strata,31 to 35,25748.0,Mar-2021,NaN,NaN


### Exploratory Data Analysis (EDA)

In [12]:
privateDf.shape

(63896, 18)

In [13]:
privateDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63896 entries, 0 to 63895
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   S/N                63749 non-null  object 
 1   Project Name       63854 non-null  object 
 2   Street Name        63620 non-null  object 
 3   Type               63619 non-null  object 
 4   Postal District    63619 non-null  object 
 5   Market Segment     63602 non-null  object 
 6   Tenure             63598 non-null  object 
 7   Type of Sale       63602 non-null  object 
 8   No. of Units       63602 non-null  float64
 9   Price ($)          63602 non-null  float64
 10  Nett Price ($)     63602 non-null  object 
 11  Area (Sqm)         63384 non-null  float64
 12  Type of Area       63602 non-null  object 
 13  Floor Level        63602 non-null  object 
 14  Unit Price ($psm)  63384 non-null  float64
 15  Date of Sale       63602 non-null  object 
 16  Area (Sqft)        218

In [14]:
# Find out mean, median, standard deviation, etc
# Data comes from 1990 to 2021
privateDf.describe()

,No. of Units,Price ($),Area (Sqm),Unit Price ($psm),Area (Sqft),Unit Price ($psf)
count,63602.000000,6.360200e+04,63384.000000,63384.00000,218.000000,218.000000
mean,1.104950,2.349265e+06,164.181954,13965.37205,3271.903670,572.110092
std,4.725047,1.242680e+07,748.609074,5495.03214,392.384892,136.581661
min,1.000000,4.000000e+04,24.000000,355.00000,1528.000000,337.000000
25%,1.000000,1.005000e+06,89.000000,10219.75000,3014.000000,458.000000
50%,1.000000,1.428000e+06,116.000000,12796.00000,3283.000000,526.500000
75%,1.000000,2.300000e+06,158.000000,16415.00000,3423.000000,659.500000
max,560.000000,9.800000e+08,87986.000000,64848.00000,5113.000000,1145.000000


## Data Preparation
- In this section, we will check if our data contains null/empty values and fill in empty data with mean values.
- We will also convert column data to correct data types.

In [15]:
# All some fields are empty
# In this case, we drop unnecessary columns and fill in missing values
privateDf.isnull().sum()

S/N                    147
Project Name            42
Street Name            276
Type                   277
Postal District        277
Market Segment         294
Tenure                 298
Type of Sale           294
No. of Units           294
Price ($)              294
Nett Price ($)         294
Area (Sqm)             512
Type of Area           294
Floor Level            294
Unit Price ($psm)      512
Date of Sale           294
Area (Sqft)          63678
Unit Price ($psf)    63678
dtype: int64

In [16]:
# Drop block and street name columns as they are irrelevant
privateDf = privateDf.drop(['S/N','Project Name','Street Name'], axis=1)
privateDf.head()

,Type,Postal District,Market Segment,Tenure,Type of Sale,No. of Units,Price ($),Nett Price ($),Area (Sqm),Type of Area,Floor Level,Unit Price ($psm),Date of Sale,Area (Sqft),Unit Price ($psf)
0,Condominium,03,RCR,99 yrs lease commencing from 2012,Resale,1.0,2000000.0,-,93.0,Strata,41 to 45,21505.0,May-2021,NaN,NaN
1,Condominium,04,RCR,99 yrs lease commencing from 1999,Resale,1.0,2300000.0,-,141.0,Strata,01 to 05,16312.0,May-2021,NaN,NaN
2,Apartment,02,CCR,99 yrs lease commencing from 2006,Resale,1.0,1251300.0,-,60.0,Strata,36 to 40,20855.0,May-2021,NaN,NaN
3,Condominium,03,RCR,99 yrs lease commencing from 2008,Resale,1.0,2530000.0,-,169.0,Strata,16 to 20,14970.0,May-2021,NaN,NaN
4,Condominium,05,OCR,956 yrs lease commencing from 1928,Resale,1.0,1150000.0,-,73.0,Strata,01 to 05,15753.0,May-2021,NaN,NaN


## Training Model - Regression
- Prob will decide which algo will fit the best for this project (might try xgboost)
- Needs to have validation dataset as well

In [ ]:
# X contains features
X = privateDf.drop(['resale_price'], axis=1)

# y contains targets to be predicted
y = privateDf['resale_price']

# 80% training data and 10% data for each test and validation data
X_train, X_testTotal, y_train, y_testTotal = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

X_validation, X_test, y_validation, y_test = train_test_split(X_testTotal, y_testTotal, test_size=0.5, random_state=42)

#### Linear Regression
We will use a simple linear regression alogrithm for this section.

In [ ]:
reg = LinearRegression()
reg.fit(X_train,y_train)

In [ ]:
mse = mean_absolute_error(y_train, reg.predict(X_train))
print("Mean Absolute Error on training data: {:.4f}".format(mse))

mse = mean_absolute_error(y_validation, reg.predict(X_validation))
print("Mean Absolute Error on validation data: {:.4f}".format(mse))

#### XGBoost
We will be using XGBoost for this section.